In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
domains_df = pd.read_csv('real_estate_domains.csv')
domains_df.head(20)

,domain
0,vk.com/public174839173
1,t.me/tweed_realtyy
2,instagram.com/tweedrealty
3,twitter.com/tweedru
4,youtube.com/channel/uca1z9uco37tm_yp9fi49rfq
5,tweed.ru
6,facebook.com/tweed.ru
7,vk.com/tweed_realty
8,pskovzhilstroi.ru
9,vk.com/domapskov


In [ ]:
# based on hand search :)
bad_starts = 'instagram facebook linkedin twitter youtube vk t wa ok viber plus zen'.split()

In [ ]:
def check_if_needless(url):
    start = url[0].split('.')[0]
    if start in bad_starts or '%' in start:
        return None
    else:
        return 'https://' + url[0]

# TODO: multiprocess it
processed_domains = domains_df.apply(check_if_needless, axis=1).dropna()
processed_domains

5                                         https://tweed.ru
8                                https://pskovzhilstroi.ru
12                                    https://dogovor25.ru
14                                         https://tg77.ru
17                                  https://capital-rf.com
                               ...                        
78953                                 https://rest2rent.ru
78958                                     https://cti33.ru
78962                                 https://veligonty.ru
78963                                   https://domkor.com
78964    https://domkor-dom.com/prodazha-kvartir-novost...
Length: 35836, dtype: object

In [ ]:
import re

def extract_main_url(s):
    m = re.search('https?://([A-Za-z_0-9.-]+).*', s)
    if m:
        return m.group(1)
    
processed_domains = processed_domains.apply(extract_main_url)
processed_domains.nunique()

17889

In [ ]:
processed_domains

5                 tweed.ru
8        pskovzhilstroi.ru
12            dogovor25.ru
14                 tg77.ru
17          capital-rf.com
               ...        
78953         rest2rent.ru
78958             cti33.ru
78962         veligonty.ru
78963           domkor.com
78964       domkor-dom.com
Length: 35836, dtype: object

In [ ]:
import requests
from threading import Lock
lock = Lock()
good  = []
badcount = 0
def url_checker(url):
    global  lock, good,  badcount
    try:
        get1 = requests.get('http://' + url)
        if get1.status_code == 200:
            res = 'http://' + url
        else: 
            get2 = requests.get('https://' + url)
            if get2.status_code == 200:
                res = 'https://' + url
            else:
                res = ''
    except:
        res = ''
    with lock:
    #  print(res, len(good), badcount)
    #if True:
      if res == "":
        badcount += 1
      else:
        good.append(res)
      if (len(good) + badcount)%128 == 0:
        print(len(good), badcount)
    return res

In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(1024) as executor:
    results = executor.map(url_checker, processed_domains)

%%time

In [ ]:
with open("filter_domains.tsv", "w") as my:
    my.write('\n'.join(list(set(good))))